## 1. Connect with drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Merge sheets

In [ ]:
import pandas as pd

# Link data 
link_data='/content/drive/MyDrive/Data/Process_Data/Data_vista.xlsx'

# Đọc file Excel vào một đối tượng ExcelFile
excel_file = pd.ExcelFile(link_data)

# Lấy danh sách tên sheet trong file
sheet_names = excel_file.sheet_names

# Tạo danh sách các dataframe dựa vào sheet_names

df_list = []


In [ ]:
for i in sheet_names:
  df = pd.read_excel(link_data, sheet_name=i)
  df_list.append(df)

In [ ]:
df_concat = pd.concat(df_list)

In [ ]:
df_concat

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,3RD
0,NaN,NaN,NaN,NaN,NaN,Mobile,NaN,NaN
1,NaN,Hs\ne,NaN,Address,Contact,Phone,NaN,NaN
2,No,/\nBlk,Unit,(2),Name,No,E-mail,NaN
3,1,T2,T2020,TP BUON MA,LE THI KIM ANH,01689 237 651,CONGCHUANHAGIAU_3000,NaN
4,NaN,NaN,2,"THUOT,",NaN,NaN,@YAHOO.COM,NaN
...,...,...,...,...,...,...,...,...
20,310.0,T2,T2340\n5,"CAU ONG\nLANH, Q. 1,",TRAN THUC NHAN,0903 606 168,CELINE_TRINH@YAHOO.CO\nM.TW,NaN
0,311.0,T2,T2340,MONTEREY,LEUNG KAI SUN,+121 3447 7249,SUNNY@LEUNG-,NaN
1,NaN,NaN,6,"PARK,",SUNNY,NaN,FAMILY.COM,NaN
2,312.0,T2,T2340\n7,"P. AN PHU, Q.2,",NGUYEN ANH NGOC,0904 869 999,ANHNGOC181099@YAHOO.C\nOM,NaN


In [ ]:
df_

In [ ]:
df_concat.isnull().sum()

Căn            1
Tên NK         1
SDT           40
Email         31
Quan hệ    10534
dtype: int64

## 3. Processing data.

### 3.1 Processin null data.

In [ ]:
df_concat.columns


Index(['Căn', 'Tên NK', 'SDT', 'Email', 'Quan hệ'], dtype='object')

In [ ]:
# Lấy các cột cần dùng.
df=df_concat[['Căn', 'Tên NK', 'SDT', 'Email']]

In [ ]:
# Check giá trị trùng lặp.
len(df['Căn'].unique())

10809

In [ ]:
# Xóa các gì trùng lặp.
df = df.drop_duplicates(subset=['Căn'], keep='first')

In [ ]:
# Thay thế các SDT không có
df=df.replace({';': None,  ';.': None,  '.': None,})

In [ ]:
# Thay thế các khách hàng không có email bằng giá trị trống.
df['Email'].fillna("Trống",inplace = True)

In [ ]:
# Check null again 
df = df.dropna(subset=['Căn','Tên NK' ,'SDT'])
df.isnull().sum()

Căn       0
Tên NK    0
SDT       0
Email     0
dtype: int64

### 3.1 Xử lý tên NK.

In [ ]:
# Xử lý Tên NK. 
df['Tên NK'] = df['Tên NK'].str.title()

### 3.2 Xử lý SDT.

In [ ]:
import re

def format_phone_numbers(phone_numbers):
    """
    Chuyển đổi các số điện thoại thành định dạng chuẩn.
    """
    # Loại bỏ các khoảng trắng và ký tự không cần thiết
    phone_numbers = re.sub('[^0-9;+]', '', phone_numbers)
    
    # Chuyển đổi các số điện thoại thành định dạng chuẩn
    formatted_numbers = []
    for phone_number in phone_numbers.split(';'):
        if phone_number.startswith('+'):
            formatted_numbers.append(phone_number)
        elif len(phone_number) == 10:
            formatted_numbers.append('+84' + phone_number[1:])
        elif len(phone_number) == 11 and phone_number.startswith('0'):
            formatted_numbers.append('+84' + phone_number[1:])
        elif len(phone_number) == 0:
            formatted_numbers = formatted_numbers
        else:
            formatted_numbers.append(phone_number)
    formatted_numbers = list(set(formatted_numbers))
    # Kiểm tra độ dài của phần tử hiện tại
    if len(formatted_numbers) < 3:
        # Thêm các giá trị 0 vào cuối phần tử để đạt độ dài bằng 5
        formatted_numbers += [''] * (3 - len(formatted_numbers))
    elif len(formatted_numbers) > 3:
        # Nếu phần tử có độ dài lớn hơn 3, thì cắt bớt đến độ dài 3
        formatted_numbers = formatted_numbers[:3]

    #return len(formatted_numbers)
    # Ghép các số điện thoại thành một chuỗi ngăn cách bằng dấu ';'
    return ';'.join(formatted_numbers)

In [ ]:
format_phone_numbers('0908638698;01234566322')

'+841234566322;+84908638698;'

In [ ]:
df_copy = df

In [ ]:
# Tạo một cột mới để chứa các số điện thoại đúng định dạng
df_copy['SDT'] = df_copy['SDT'].apply(format_phone_numbers)

In [ ]:
df_copy[['SDT_1', 'SDT_2', 'SDT_3']] = df_copy['SDT'].str.split(';',expand=True)

In [ ]:
df_copy.to_excel('/content/drive/MyDrive/Data/Raw_data/Vinhiomes/test_1.xlsx', index=False)